## US map of confirmed cases 

In this notebook we would like to study the number of confirmed cases per state in the US.

We will also correlate the spread of the desease with the size of the population per state

In [7]:
import pandas as pd
import os
import utils.preprocess_data as ppd
import dash 
import plotly.express as px
import json 

In [8]:
df_usa = ppd.get_states_data(dir_name="../data/covid",
                         dir_name_zipcode_data="../data/us_meta")

['NE (From Diamond Princess)', 'CA (From Diamond Princess)', 'TX (From Diamond Princess)', 'Unassigned Location (From Diamond Princess)', 'Grand Princess Cruise Ship', 'D.C.', 'Diamond Princess', 'Grand Princess', 'PR', 'U.S.', 'Guam', 'Virgin Islands', 'United States Virgin Islands', 'US', 'Wuhan Evacuee', 'American Samoa', 'Northern Mariana Islands', 'Recovered']
{'Acmar': 'AL', 'Adamsville': 'TN', 'Adger': 'AL', 'Keystone': 'WV', 'New site': 'MS', 'Alpine': 'WY', 'Arab': 'AL', 'Baileyton': 'TN', 'Bessemer': 'PA', 'Zcta 35022': 'AL', 'Hueytown': 'AL', 'Blountsville': 'AL', 'Bremen': 'KY', 'Brent': 'AL', 'Brierfield': 'AL', 'Brookside': 'AL', 'Calera': 'OK', 'Centreville': 'MS', 'Chelsea': 'VT', 'Coosa pines': 'AL', 'Clanton': 'AL', 'Zcta 35046': 'AL', 'Cleveland': 'WI', 'Columbiana': 'OH', 'Cook springs': 'AL', 'Crane hill': 'AL', 'Cropwell': 'AL', 'Cullman': 'AL', 'Zcta 35058': 'AL', 'Docena': 'AL', 'Dolomite': 'AL', 'Dora': 'NM', 'Empire': 'OH', 'Fairfield': 'WA', 'Coalburg': 'AL',

In [9]:
fig = px.choropleth(df_usa, 
                    locations=df_usa.Code, 
                    locationmode = "USA-states",
                    color="Confirmed", 
                    color_continuous_scale="Peach",
                    animation_frame="Date"
                   )

fig.update_layout(
    title_text = 'US Spread of Coronavirus',
    title_x = 0.5,
    geo_scope='usa',
    geo=dict(
        showframe = False,
        showcoastlines = False,
    ))
    
fig.show()